In [49]:
from datasets import SmallDataset
from models import get_model
from utils.transforms import get_transform
import utils.engine as engine

import math
import sys

import torch

In [46]:
model = get_model(num_classes=5)
dataset = SmallDataset("/home/kamranzolfonoon/eagle-test-bucket", "/home/kamranzolfonoon/eagle-test-bucket/image_labels.json", get_transform(train=False))

In [47]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [48]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [18]:
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# prediction = model(x)
# print(prediction[0]["labels"])

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4,
        3, 4, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4,
        4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4,
        3, 3, 4, 3])


In [54]:

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=collate_fn)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
loss_over_run = []

for epoch in range(num_epochs):
    loss_over_epoch = []
    model.train()

    print("Epoch: ", epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=False):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        loss_value = losses
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict)
            sys.exit(1)

        loss_over_epoch.append(loss_value.item())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    average_loss = sum(loss_over_epoch) / len(loss_over_epoch)
    loss_over_run.append(average_loss)
    print(f"EPOCH: {epoch}, LOSS: {average_loss}")
    lr_scheduler.step()

print("That's it!")

Epoch:  0
EPOCH: 0, LOSS: 0.9078442234668374
Epoch:  1
EPOCH: 1, LOSS: 0.6412049821914642


AttributeError: 'NoneType' object has no attribute 'step'

In [59]:
model.eval()
img, target = dataset[2]
print(target)
img = img.to(device)
predictions = model([img])
print(predictions[0]["boxes"])
print(predictions[0]["labels"])

{'boxes': tensor([[ 503.,  253., 1012.,  591.],
        [ 471.,  448.,  526.,  496.],
        [ 510.,  433.,  562.,  454.]]), 'labels': tensor([0, 1, 1])}
tensor([[ 476.6903,  407.3584, 1026.6876,  628.5165],
        [ 422.3098,  263.6180, 1034.9056,  590.6732],
        [ 659.8234,  359.4484, 1068.8989,  588.1248],
        [ 459.3795,  355.6681,  880.9531,  578.1329],
        [ 638.2814,  243.7959, 1131.8130,  680.9591],
        [ 387.3619,  332.3712,  933.1053,  714.7035],
        [ 972.7573,  495.0476, 1076.5699,  553.7318],
        [ 489.7808,  438.8138, 1232.0000,  718.0408],
        [ 678.5583,  275.1361, 1028.0959,  520.9781],
        [ 493.6762,   70.4027,  971.4437,  669.9131],
        [ 973.5790,  491.1417, 1103.7931,  586.8069],
        [ 552.8942,  478.3871, 1085.5641,  670.7880],
        [ 368.5137,  354.7232,  758.3600,  534.1495],
        [ 470.1024,  321.1612,  692.7694,  609.9719],
        [ 459.9166,  421.1677,  649.4531,  560.4008],
        [  17.1002,  428.7850,  964